In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import timm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# ✅ DeiT 분류 모델 정의
class DeiTClassifier(nn.Module):
    def __init__(self):
        super(DeiTClassifier, self).__init__()
        self.backbone = timm.create_model(
            'deit_base_patch16_224', pretrained=True, num_classes=3
        )

    def forward(self, x):
        return self.backbone(x)

# ✅ 사용자 정의 데이터셋
class ScrapClassificationDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, label_encoder=None):
        self.data = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = label_encoder or LabelEncoder()
        self.data['class_idx'] = self.label_encoder.fit_transform(self.data['weight_class'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx]['filename'])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.data.iloc[idx]['class_idx'], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# ✅ 평가 함수
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds) * 100
    f1 = f1_score(all_labels, all_preds, average='macro')
    return acc, f1

# ✅ 경로 설정
csv_path = r"C:\Users\ast\Documents\project\weight.xlsx"
img_dir = r"C:\Users\ast\Documents\project\dataset"

# ✅ 데이터 로드 및 전처리
df = pd.read_excel(csv_path)
df['weight_class'] = pd.qcut(df['KG'], q=3, labels=['light', 'medium', 'heavy'])
df['filename'] = df['No'].apply(lambda x: f"{int(x):03d}.JPG")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ K-Fold 학습
kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    print(f"\n📂 Fold {fold+1}")

    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]
    label_encoder = LabelEncoder()

    train_dataset = ScrapClassificationDataset(train_df, img_dir, transform, label_encoder)
    val_dataset = ScrapClassificationDataset(val_df, img_dir, transform, label_encoder)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = DeiTClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)

    epochs = 10
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    acc, f1 = evaluate_model(model, val_loader, device)
    print(f"✅ Fold {fold+1} - Accuracy: {acc:.2f}%, F1 Score: {f1:.4f}")
    fold_results.append((acc, f1))

# ✅ 전체 평균 출력
avg_acc, avg_f1 = np.mean(fold_results, axis=0)
print("\n📊 Average Results over 5 folds:")
for i, (acc, f1) in enumerate(fold_results):
    print(f"Fold {i+1}: Accuracy = {acc:.2f}%, F1 Score = {f1:.4f}")
print(f"\nOverall Average: Accuracy = {avg_acc:.2f}%, F1 Score = {avg_f1:.4f}")



📂 Fold 1


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]